__Group names:__ [Boram Lee, Steven Ang, Juho Immonen, Yana Berezina]

__Group SUNET IDs:__ [06932480]

# **Before you get started, please ensure that you have installed the following packages: remotes, vctrs (version 0.6.1), and tzdb (version 0.4.0). Instructions are located in the canvas where you downloaded this assignment. Remember to change your kernel to R**

# Lecture 10 Overview

Today, we will apply what we've learned in class thus far, using resources and data available through Stanford.

At the end of today's class, you should be able to:
1. use SQL to query STARR-OMOP data to build a cohort of patients, and
2. use R and SQL to do a descriptive analysis.

We will ask each group to submit ONE notebook through Canvas. Please write the names and SUNET IDs of your group members in the cell above this one.

If you and your group have questions, just ask! A teaching assistant will be happy to help you.

# Table of Contents

__0. Prepare your R session to query STARR-OMOP, and run a query.__ (~5 minutes) <br>

__1. Walk through a completed example of building a cohort.__ (~20 minutes) <br>
    We show you how to find people with chronic kidney disease, defined as the presence of:<br>
- the ICD-9 code `585.9` (indicating chronic kidney disease) at least once, and
- at least one abnormally high creatinine measurement ($\gt 1.4$). <br>

__2. With your group, build a cohort of people with type II diabetes.__(~ 20 minutes)<br>
We define type II diabetes as the presence of:<br>
- the ICD-9 code `250.00` (indicating diabetes type II or unspecified), and
- at least one abnormally high HbA1c measurement ($\geq 8$).<br>

__3. With your group, perform a descriptive study of the cohort of patients you found with type II diabetes.__(~ 15+ minutes) <br>
If you can't finish this in class, that's OK! You can finish this with your group after class if you'd like!

__4. Excited to learn more?__<br>
Here are some extra queries to do with your group or on your own, to learn more about how to perform a descriptive analysis on STARR-OMOP! (This section is not required for full credit.)

As you go through this notebook, look out for sections labeled ___Reflection Question___! Answering these questions will give you a chance to think about potential pitfalls in the end-to-end process of defining a cohort and performing an analysis.

# 0. Get to know SQL and the OMOP data model

### Data: STARR-OMOP, Confidential

To access data, we will be working on a JupyterLab environment set up on Stanford's shared computing platform specifically designed for high risk data, operated by the Stanford Research Computing Center.

We will be using a dataset referred to as STARR-OMOP. The name STARR-OMOP has two components. "STARR" stands for STAnford Research Repository, and refers to the Stanford EHR data we will use. As we know from class, "[OMOP](https://www.ohdsi.org/data-standardization/the-common-data-model/)" stands for the Observational Medical Outcomes Partnership, and it refers to the _data model_ in which the STARR data is stored.

Today, we're working with a "lite" version (without clinical notes) of STARR-OMOP that has been deidentified, but is still considered high risk and thus confidential.

[Here](https://med.stanford.edu/starr-omop/access.html#documentation) is a list of excellent resources from Stanford Medicine's Research IT department.

The most important resource for today will likely be Stanford Medicine's [STARR-OMOP data dictionary](https://docs.google.com/spreadsheets/d/1BW1tRRml5E20ivoA5c8DhfKeuP449cBECkkpKoycwgk/). If you have questions about data tables and their relationships, check here first! Also, note that the STARR-OMOP dataset contains some extra columns which are not strictly part of the CDM definition, but have been added for increased source/patient traceability.

### Tools

As with the rest of the course, we will be working in R. STARR-OMOP is stored in Google BigQuery, and we will query it using SQL; fortunately, there is an R package (`bigrquery`) that allows us to do this without leaving the R environment.

(Though we will use R today, there are equivalent Python packages for querying data that is stored in BigQuery.)
### What is SQL?

SQL is an abbreviation for Structured Query Language, developed to manage data stored in a relational database. With SQL, and you can define and extract subsets of data, and make joins across data tables; working in SQL may feel familiar to manipulating and joining data frames in pandas (Python) and dplyr/data.table (R).

### Setup

To query tables using R, we first need to load packages and set up our system environment:

In [1]:
###############################################################
# This cell is very important for the workshop to run!
###############################################################

library(bigrquery)  # to query STARR-OMOP (stored in BigQuery) using SQL
library(dplyr)      # to analyze data in R, and do our descriptive analysis

# We have credentials to access data, but we need to tell our system environment about them.
# Fill in your own SUNet ID here!
MY_USERNAME <- "steveang"

# This sets our system variables:
MY_CREDENTIALS <- paste0("/home/", MY_USERNAME, "/.config/gcloud/application_default_credentials.json")
Sys.setenv(GOOGLE_APPLICATION_CREDENTIALS = MY_CREDENTIALS)
Sys.setenv(GCLOUD_PROJECT = "som-nero-nigam-bmi215")
gargle::credentials_app_default();

# To access the data, we have a "project" where our data access permissions have been defined.
# You don't need to worry about this today - but it will appear in all of our queries.
project_name <- "som-nero-nigam-bmi215"


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




NULL

Let's do our first query: _how many people are in the lite de-id version of STARR-OMOP?_

In OMOP, the `person` table has one row for every person in the data ([reference](https://github.com/OHDSI/CommonDataModel/wiki/PERSON)). To see how many people are in our dataset, we will count the rows of the `person` table .

Our query will be:

```
SELECT
    COUNT(*)
  FROM
    som-rit-phi-starr-prod.starr_omop_cdm5_confidential_lite_latest.person
```

Note that we refer to the `person` table using its full path:

```som-rit-phi-starr-prod.starr_omop_cdm5_confidential_lite_latest.person```

We will do this every time we query any table! We refer to each table by its full name in the form `project_name.dataset_name.table_name`. In this case, the project name is `som-rit-phi-starr-prod`, and the dataset name is `starr_omop_cdm5_confidential_lite_latest`.

Let's run this query!

In [2]:
sql = "
SELECT
    COUNT(*)
FROM
    som-rit-phi-starr-prod.starr_omop_cdm5_confidential_lite_latest.person
"

# Query the data:
tb <- bq_project_query(project_name, sql)

# And download the result of the query, as an R data.frame:
bq_table_download(tb)

f0_
<int>
4030408


This returns the number of patients in the de-id lite version of the STARR data.

The path `som-rit-phi-starr-prod.starr_omop_cdm5_confidential_lite_latest` is cumbersome to type (accurately) in every query! We'll use the following helper function to reduce errors:

In [3]:
###############################################################
# This cell is very important for the workshop to run!
###############################################################

# format.query takes in a string containing a SQL query.
# It replaces all instances of "project_name" with "som-rit-phi-starr-prod".
# And it replaces all instances of "dataset_name" with "starr_omop_cdm5_confidential_lite_latest".
format.query <- function(sql.string){
    clean.string <- gsub("project_name", "som-rit-phi-starr-prod", sql.string)
    clean.string <- gsub("dataset_name", "starr_omop_cdm5_confidential_lite_latest", clean.string)
    return(clean.string)
}

# Here is an example of how it works:
cat(format.query("
SELECT
    COUNT(*)
FROM
    project_name.dataset_name.person
"))


SELECT
    COUNT(*)
FROM
    som-rit-phi-starr-prod.starr_omop_cdm5_confidential_lite_latest.person


In [4]:
# And here is our query using our new helper function:
sql = format.query("
SELECT
    COUNT(*)
FROM
    project_name.dataset_name.person
")

tb <- bq_project_query(project_name, sql)
count.total_patients <- bq_table_download(tb)
count.total_patients

f0_
<int>
4030408


In [5]:
# What else is in the person table?
# Let's look at the first 5 rows.
sql = format.query("
SELECT
    *
FROM
    project_name.dataset_name.person
LIMIT 5
")

tb <- bq_project_query(project_name, sql)
first.five.patients <- bq_table_download(tb)
first.five.patients

person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_DATETIME,race_concept_id,ethnicity_concept_id,location_id,provider_id,⋯,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id,trace_id,unit_id,load_table_id
<int>,<int>,<int>,<int>,<int>,<dttm>,<int>,<int>,<int>,<int>,⋯,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>
31215478,8507,1960,1,8,1960-01-08,0,0,1950215,NA,⋯,SAA1dc4f76,2 |,8507,|,0,|,0,NA,NA,NA
64779860,8532,1964,1,8,1964-01-08,0,0,5056883,NA,⋯,SAA3dc7654,1 |,8532,|,0,|,0,NA,NA,NA
29964803,8532,1968,1,8,1968-01-08,0,0,2334376,NA,⋯,SAA1c93a03,1 |,8532,|,0,|,0,NA,NA,NA
53202542,8532,1972,1,8,1972-01-08,0,0,5214373,NA,⋯,SAA32bce6e,1 |,8532,|,0,|,0,NA,NA,NA
32556396,8507,1976,1,8,1976-01-08,0,0,4914840,NA,⋯,SAA1f0c56c,2 |,8507,|,0,|,0,NA,NA,NA


# 1. Building a cohort: an example

Let's find patients with chronic kidney disease, defined as the presence of:
1. the ICD-9 code `585.9` (indicating chronic kidney disease) at least once, __and__
2. at least one abnormally high creatinine measurement ($\gt 1.4$).

To find these patients, our approach will consist of three steps:
1. find patients who meet criterion 1,
2. find patients who meet criterion 2, and
3. use an _inner join_ to find the intersection of (patients who meet criterion 1) and (patients who meet criterion 2).

As we go through these steps together, we'll learn about some of the tables in the OMOP data structure.

## 1.1 Requirement 1: Patients With ICD-9 Code `585.9`

### 1.1.1 The Condition Occurrence Table

Let's do a couple of examples, to practice SQL and look at the OMOP data structure.

We've seen the `person` table. But how do we access data from other parts of the medical record?

One place to look is in the `condition_occurrence` table. In this table, each row contains a person ID, a condition, and a start time for that condition, as well as other details about the record including the visit it is linked to, and the provider who recorded it. (Each person will likely have multiple rows in this table.)

The OHDSI documentation describes a _condition_ as follows.

> Conditions are records of a Person suggesting the presence of a disease or medical condition stated as a diagnosis, a sign, or a symptom, which is either observed by a Provider or reported by the patient. Conditions are recorded in different sources and levels of standardization, for example:
>
> - Medical claims data include diagnoses coded in Source Vocabularies such as ICD-9-CM that are submitted as part of a reimbursement claim for health services
> - EHRs may capture Person conditions in the form of diagnosis codes or symptoms


In [6]:
# What are the first 10 rows of the condition_occurrence table?
sql = format.query("
SELECT
    person_id, condition_concept_id, condition_start_date
FROM
    project_name.dataset_name.condition_occurrence
LIMIT 10
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

person_id,condition_concept_id,condition_start_date
<int>,<int>,<date>
96950371,0,2023-03-31
64785444,0,2024-03-24
32160232,0,2024-04-02
85694441,0,2024-10-16
31815977,0,2024-01-26
111886129,0,2024-04-10
45692803,0,2023-04-30
111515069,0,2024-10-15
97077378,0,2023-01-14


We see a person id (which links to the `person` table), a start date, and a `condition_concept_id`. The `condition_concept_id` is not easy to interpret!

The `condition_concept_id`s map back to the `concept` table, which tells us more about what each condition means. For example, let's look at the `concept` table to see what `condition_concept_id` = `434547` refers to.

In [7]:
sql = format.query("
SELECT
    concept_id, concept_name, domain_id, concept_code, vocabulary_id, concept_class_id
FROM
    project_name.dataset_name.concept
WHERE concept_id = 434547
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

concept_id,concept_name,domain_id,concept_code,vocabulary_id,concept_class_id
<int>,<chr>,<chr>,<chr>,<chr>,<chr>
434547,Complication of surgical procedure,Condition,88797001,SNOMED,Clinical Finding


So, `concept_id` 434547 is **complication of a surgical procedure**, as defined by code 88797001 in the SNOMED-CT ontology.

We can find all patients who had this code - and the time(s) they were labeled with this code - with a single query to the `condition_occurrence` table:

In [8]:
# Find 10 rows of the condition_occurrence table with SNOMED-CT code 88797001.
# From our query above, we know that SNOMED-CT code 88797001 corresponds to concept_id 434547.
sql = format.query("
SELECT
    person_id, condition_concept_id, condition_start_date
FROM
    project_name.dataset_name.condition_occurrence
WHERE
    condition_concept_id = 434547
LIMIT 10
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

person_id,condition_concept_id,condition_start_date
<int>,<int>,<date>
31395674,434547,2022-05-05
30147046,434547,2021-07-06
32517726,434547,2020-05-04
31550445,434547,2022-11-29
63906581,434547,2024-07-26
32232227,434547,2024-01-09
46020724,434547,2021-04-26
31582112,434547,2018-11-18
30737701,434547,2019-01-03


### 1.1.2 Patients with ICD-9 Code 585.9

Now, we want to find all patients with the ICD-9 code `585.9`.

Naively, here is how we could look up the `concept_id` for the ICD-9 code for chronic kidney disease, `585.9`.

In [9]:
sql = format.query("
SELECT
    concept_id
FROM
    project_name.dataset_name.concept concept
WHERE
    concept.concept_code = '585.9'
    AND concept.vocabulary_id = 'ICD9CM'
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

concept_id
<int>
44836035


Let's count the number of unique patients in the `condition_occurrence` table that have this `concept_id`:

In [10]:
# Count the number of patients who are labeled with ICD-9 code 585.9 at least once.
sql = format.query("
SELECT
    COUNT(DISTINCT person_id)
FROM
    project_name.dataset_name.condition_occurrence
WHERE
    condition_concept_id = 44836035
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
0


We found 0 patients with chronic kidney disease! Does this make sense?

___Reflection Question:___ What could we do to help decide whether it "makes sense" to find 0 patients with chronic kidney disease in our data?<br>
___Answer:___ It does make sense if we think about chronic kidney disease ontology. There are many ontologies for chronic kidney disease and ICD-9 code 585.9 just one of it. It might be a case that this database use other ontologies for chronic kidney disease and not 585.9.

So what happened? The `concept_id` we found corresponds to the ICD-9 code `585.9`. But there are many ontologies, and so there are many ways to represent "chronic kidney disease". (For example, in the SNOMED-CT ontology, "chronic kidney disease" is represented as code [709044004](https://snomedbrowser.com/Codes/Details/709044004).) The `condition_occurrence` table does not use the ICD-9 ontology as its source of `concept_id`s - in this case, it uses SNOMED-CT instead.

The `concept_relationship` table tells us how concepts have been mapped across ontologies, and we can use this table to map the ICD-9 code `585.9` to the `concept_id` of the corresponding SNOMED-CT code:

In [11]:
sql = format.query("
SELECT
    concept_id_1, vocabulary_id, concept_code, relationship_id, concept_id_2
FROM project_name.dataset_name.concept concept
JOIN project_name.dataset_name.concept_relationship concept_relationship ON concept_relationship.concept_id_1 = concept.concept_id
WHERE
    concept_code = '585.9'
    AND vocabulary_id = 'ICD9CM'
    AND relationship_id = 'Maps to'
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

concept_id_1,vocabulary_id,concept_code,relationship_id,concept_id_2
<int>,<chr>,<chr>,<chr>,<int>
44836035,ICD9CM,585.9,Maps to,46271022


The `concept_id_1` 44836035 (for ICD-9 code `585.9`) is mapped to `concept_id_2` = 46271022.

To find patients with ICD-9 code `585.9`, we use the `concept_id` 46271022:

In [12]:
# Count the number of patients who are labeled with ICD-9 code 585.9 at least once.
sql = format.query("
SELECT
    COUNT(DISTINCT person_id)
FROM project_name.dataset_name.condition_occurrence
WHERE condition_concept_id = 46271022
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
47285


___Reflection Question:___ Is there always just one SNOMED-CT concept for every ICD-9 code? Do you see any potential problems with this method of storing data?<br>
___Answer:___ No, there isn't always a one-to-one relationship between ICD-9 codes and SNOMED CT concepts becuase SNOMED represents the clinical concept more generally while ICD-9 breaks it down by specific anatomical location and episode type.  Some potential problems are: Query complexity, Daya consistency... etc

## 1.2 Requirement 2: Patients With One Or More Abnormal Creatinine Measurements

### 1.2.1 The Measurements Table
In addition to "conditions", patients can also have "measurements". While "conditions" are often expressed using the SNOMED-CT and ICD ontologies, "measurements" are often expressed using the LOINC ontology.

___Reflection Question:___ What does LOINC stand for? <br>
___Answer:___ Logical Observation Identifiers Names and Codes

For example, the LOINC code for a measurement of creatinine is `2160-0`.

What is its `concept_id` in our data?

In [13]:
sql = format.query("
SELECT
    concept_relationship.concept_id_2
FROM
    project_name.dataset_name.concept
JOIN
    project_name.dataset_name.concept_relationship
ON
    concept_relationship.concept_id_1 = concept.concept_id
WHERE
    concept_code = '2160-0'
    AND vocabulary_id = 'LOINC'
    AND relationship_id = 'Maps to'
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

concept_id_2
<int>
3016723


To find patients who had their creatinine measured - and the values of those measurements - we look in the `measurement` table.

In [14]:
# Find 15 rows of the condition_occurrence table with LOINC code 2160-0.
# From our query above, we know that LOINC code 2160-0 corresponds to concept_id 3016723.
sql = format.query("
SELECT
    person_id, value_as_number, measurement_datetime
FROM
    project_name.dataset_name.measurement
WHERE
    measurement_concept_id = 3016723      -- creatinine measurement
LIMIT 15
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

person_id,value_as_number,measurement_datetime
<int>,<dbl>,<dttm>
30456344,0.83,2018-07-06 08:10:00
31081917,1.70,2013-08-20 15:35:00
31383965,1.20,2018-07-29 09:04:00
30739827,0.60,2016-03-12 09:12:00
30453398,0.88,2019-06-02 10:20:00
32262726,1.40,2013-08-27 13:35:00
31986480,0.51,2017-07-28 06:58:00
30075617,0.95,2018-05-10 10:10:00
31029654,8.63,2018-08-28 14:51:00


Ack - many of those values are null! How many are null?

In [15]:
sql = format.query("
SELECT
    COUNT(*)
FROM
    project_name.dataset_name.measurement
WHERE
    measurement_concept_id = 3016723      -- creatinine measurement
    AND value_as_number IS NULL           -- measurement is null
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
691734


How many total creatinine measurements are there?

In [16]:
sql = format.query("
SELECT
    COUNT(*)
FROM
    project_name.dataset_name.measurement
WHERE
    measurement_concept_id = 3016723      -- creatinine measurement
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
13541204


___Reflection Question:___ What percent of creatinine measurements are null? Can you think of a reason or two why a creatinine value might be missing?<br>
___Answer:___ 5%. The possible reasons why the value is missing: 1. Test was ordered but not complete. 2. Technical reasons like the result stored in different system and not linked to the database we used.  Result stored in unstructured format. or Data lost.

### 1.2.2 Patients With One Or More Abnormally High Creatinine Measurements

According to our phenotype definition, we wish to find patients with a creatinine measurement above 1.4:

In [17]:
sql = format.query("
SELECT
    person_id, value_as_number, measurement_datetime
FROM
    project_name.dataset_name.measurement
WHERE
    measurement_concept_id = 3016723      -- creatinine measurement
    AND value_as_number > 1.4             -- value above 1.4
LIMIT 10
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

person_id,value_as_number,measurement_datetime
<int>,<dbl>,<dttm>
31418271,6.81,2024-02-05 09:02:00
31194219,2.18,2017-03-19 20:00:00
69262327,1.43,2022-03-29 18:01:00
31643474,1.88,2006-09-29 03:30:00
31832362,1.72,2023-07-27 04:51:00
32457105,12.70,2010-01-01 04:50:00
30181623,2.90,2012-09-29 17:00:00
32097813,3.20,2014-10-19 09:39:00
31496690,1.80,2018-06-27 11:09:00


How many patients have one or more creatinine measurements above 1.4?

In [18]:
sql = format.query("
SELECT
    COUNT(DISTINCT person_id)
FROM
    project_name.dataset_name.measurement
WHERE
    measurement_concept_id = 3016723      -- creatinine measurement
    AND value_as_number > 1.4             -- value above 1.4
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
122717


If we wanted to count the patients with at least one non-null measurement, we could use `IS NOT NULL`:

In [19]:
sql = format.query("
SELECT
    COUNT(DISTINCT person_id)
FROM
    project_name.dataset_name.measurement
WHERE
    measurement_concept_id = 3016723      -- creatinine measurement
    AND value_as_number IS NOT NULL       -- value not null
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
1220053


___Reflection Question:___ Of people who had at least one non-null creatinine measurement, what fraction had a measurement above 1.4?<br>
___Answer:___ 10%

## 1.3 Patients with ICD-9 585.9 and an Abnormal Creatinine Measurement

We'd like to find all patients with at least one measurement of creatinine that is above 1.4, and at least one diagnosis of chronic kidney disease (ICD-9 code `585.9`). We've found patients meeting each of those criteria; now we'd like to combine our results by finding patients who meet __both__ criteria.

To do this, we'll have to look at the `measurement` table and the `condition_occurrrence` table together.

___Reflection Question:___ How many patients did we find with ICD-9 code `585.9`? How many patients did we find with at least one measurement of creatinine above $1.4$? At most, how many patients can we find who meet both criteria?<br>
___Answer:___

Here is how we would:
1. find the people who are in the `measurement` table and the `condition_occurrence` table, and
2. restrict to those who have a creatinine measurement above 1.4, and the ICD-9 code 585.9.

In [20]:
sql = format.query("
SELECT DISTINCT
    measurement.person_id
FROM
    project_name.dataset_name.measurement
JOIN
    project_name.dataset_name.condition_occurrence
ON
    measurement.person_id = condition_occurrence.person_id
WHERE
    measurement.measurement_concept_id = 3016723     -- creatinine measurement
    AND measurement.value_as_number > 1.4            -- measurement value > 1.4
    AND condition_occurrence.condition_concept_id = 46271022  -- ICD9 code 585.9
")

tb <- bq_project_query(project_name, sql)
person_ids.high_creatinine_and_ckd <- bq_table_download(tb)

head(person_ids.high_creatinine_and_ckd)

person_id
<int>
29987323
30524529
30196806
32399754
31181643
30857249


In [21]:
nrow(person_ids.high_creatinine_and_ckd)

[1] 32482

___Reflection Question:___ How many patients have this phenotype? (You may wish to use `nrow` in R.)<br>
___Answer:___ 32482

___Reflection Question:___ How many unique patients were in the `person` table? What fraction of those patients met _both_ criteria? What do you think would happen if we used more criteria? How might this impact studies using EHR data?<br>
___Answer:___ There are 4030408 unique patient in the `person`table. If we add more criteria, then the number of unique patient might go down further

# 2. Building a Cohort: Your Turn

### 2.0 Phenotype definition
We'd like our cohort to consist of people with type II diabetes. We've seen in class that diabetes is particularly challenging to phenotype - for the purpose of today's exercise, we'll keep the phenotype simple.

We consider a person to have type II diabetes when:
1. they have been given the ICD-9 code `250.00` (indicating diabetes type II or unspecified) at least once, and
2. they have at least one HbA1c measurement $\geq 8$.

(Note that the above is not a robust phenotype definition! That is, we should not feel confident that we have captured a significant fraction of those with type II diabetes, nor that everyone who meets this phenotype definition has type II diabetes.)

To build the cohort of patients with type II diabetes, we will walk through the following steps:
1. Find the concept ID for the ICD-9 code `250.00`.
2. Find the concept ID for the LOINC code `4548-4` (HbA1c measurement).
3. Use these concept IDs to find patients who fit our phenotype definition.

### 2.1 Concept IDs
First, find the `concept_id` corresponding to ICD-9 code `250.00`, "Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled."

You may choose to use our example for CKD.

In [22]:
############################################################
##### FILL IN YOUR SOLUTION HERE
############################################################
sql = format.query("
SELECT
    concept_relationship.concept_id_2
FROM
    project_name.dataset_name.concept concept
JOIN
    project_name.dataset_name.concept_relationship concept_relationship
ON
    concept_relationship.concept_id_1 = concept.concept_id
WHERE
    concept_code = '250.00'
    AND vocabulary_id = 'ICD9CM'
    AND relationship_id = 'Maps to'
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

concept_id_2
<int>
4193704


As we build our cohort, it can be useful to track how many patients we have at each step. How many distinct `person_id`s have at least one instance of the ICD-9 code `250.00`?

In [23]:
############################################################
##### FILL IN YOUR SOLUTION HERE
############################################################
sql = format.query("
SELECT
    COUNT(DISTINCT person_id)
FROM
    project_name.dataset_name.condition_occurrence
WHERE
    condition_concept_id = 4193704
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
150727


Now, find the `concept_id` for HbA1c (LOINC code `4548-4`). You may use our example for creatinine.

In [24]:
############################################################
##### FILL IN YOUR SOLUTION HERE
############################################################
sql = format.query("
SELECT
    concept_relationship.concept_id_2 AS concept_id
FROM
    project_name.dataset_name.concept
JOIN
    project_name.dataset_name.concept_relationship concept_relationship
ON
    concept_relationship.concept_id_1 = concept.concept_id
WHERE
    concept_code = '4548-4'
    AND vocabulary_id = 'LOINC'
    AND relationship_id = 'Maps to'
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

concept_id
<int>
3004410


### 2.2 Measurements

How many people have at least one HbA1c measurement greater than or equal to $8$\? (You may wish to use our example for creatinine from Section 1.)

In [25]:
############################################################
##### FILL IN YOUR SOLUTION HERE
############################################################
sql = format.query("
SELECT
    COUNT(DISTINCT person_id)
FROM
    project_name.dataset_name.measurement
WHERE
    measurement_concept_id = 3004410
    AND measurement.value_as_number >= 8
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
64273


### 2.3 Putting it all together

Now we have everything we need! Count the number of distinct `person_id`s for people who have:
- at least one HbA1c measurement $\geq 8$, and
- at least one label of ICD-9 code `250.00`.

This should be similar to the query where we found patients with the ICD-9 code for CKD, `585.9`, and a creatinine measurement above $1.4$.

In [26]:
############################################################
##### FILL IN YOUR SOLUTION HERE
############################################################
sql = format.query("
SELECT
    COUNT(DISTINCT measurement.person_id)
FROM
    project_name.dataset_name.measurement measurement
JOIN
    project_name.dataset_name.condition_occurrence condition
ON
    measurement.person_id = condition.person_id
WHERE
    measurement.measurement_concept_id = 3004410
    AND measurement.value_as_number >= 8
    AND condition.condition_concept_id = 4193704
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
42066


# 3. Do a descriptive analysis

Now that we have a set of patients who meet our cohort defintion, it is useful to gather some desciptive measurements to get an idea of what these patient medical records look like.  It is always a good idea to thoroughly explore your data to get a sense of how the data is strucutred, whether there exist outliers, and to sanity check your cohort definition.

We won't have time for an exhaustive descriptive analysis, but we'll be able to examine and summarize information regarding patient demographics, and common comorbidities they may have.  If you get to it (and if you so desire) you may also complete part 4 of this workshop which looks at labs that are commonly ordered for patients in your cohort and summarizes their results.

## 3.1 Extract and stratify by patient demographics
Demographic information can be found in the `person` table in the STARR OMOP dataset.  One common critique of RCT based evidence is that enrollee demographics are highly enriched for middle aged white men - calling into question the external validity of results, especially for individuals of underpresented groups. Real world observational data, in many cases, may not be much better in this regard. In this section you'll use the cohort we've defined above for type II diabetes and summarize the demographic distribution of the data.

### 3.1.1 Familiarize yourself with Common Table Expression (CTEs)
It is often useful to modularize your SQL query into Common Table Expressions (CTEs), especially when performing more complex queries that require multiple JOINs.  Below, we first show a sample query that counts the number of patients in OMOP with a high creatinine value.

We then give an example that uses a CTE that stores this list of `person_ids` in a result set that can be accessed in downstream components of a more complicated query.

This example may be useful to you as you develop queries for your descriptive analysis.

In [27]:
# The below query finds the number of people with a creatinine measure above 1.4 in the OMOP dataset
sql = format.query("
SELECT
   COUNT(DISTINCT person_id)
FROM
    project_name.dataset_name.measurement
WHERE
    measurement_concept_id = 3016723
    AND value_as_number > 1.4
")
tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
122717


Now, we use a CTE to abstract away the above query and define it in a temporary result set called high_creatinine. We then join to the death table to see how many of these patients are still alive (as of 10-24-2024) according to STARR OMOP.

In [28]:
# -- WITH [expression_name] AS ([SQL EXRESSION]) defines the CTE.
sql = format.query("
WITH high_creatinine AS (
SELECT
    person_id, value_as_number, measurement_datetime
FROM
    project_name.dataset_name.measurement
WHERE
    measurement_concept_id = 3016723
    AND value_as_number > 1.4
)

-- Joins high_creatine (defined above) to the death table by person ID and
-- counts number of people w/o a record of death.
SELECT
    COUNT(DISTINCT  hc.person_id)
FROM
    high_creatinine hc
LEFT JOIN
    project_name.dataset_name.death d
USING(person_id)
WHERE
    d.death_DATE IS NULL
")
tb <- bq_project_query(project_name, sql)
bq_table_download(tb)

f0_
<int>
103586


### 3.1.2 Count the number of male and female patients in our cohort
Create a CTE that stores the person_ids of the individuals we've phenotyped as having Type II diabetes.  Then use this CTE to query against the `person` and `concept` tables to extract information about these people's sex.

Read the query into an R dataframe, stratify by sex and count the number of unique patients with said sex.

For your convenience, we've provided much of the skeleton code you'll need to construct this query.  Anything in a bracket `[]` indicates things you'll need to replace with table/column definitions or code.

In [29]:
############################################################
##### YOUR SOLUTION GOES HERE
############################################################
sql = format.query("
WITH cohort AS (
    SELECT 
        DISTINCT measurement.person_id
    FROM 
        project_name.dataset_name.measurement measurement
    JOIN 
        project_name.dataset_name.condition_occurrence condition
    ON 
        measurement.person_id = condition.person_id
    WHERE
        measurement.measurement_concept_id = 3004410
        AND measurement.value_as_number >= 8
        AND condition_concept_id = 4193704 
)


SELECT
    c.person_id, concept.concept_name
FROM
    cohort c
INNER JOIN
    project_name.dataset_name.person
USING
    ( person_id )
INNER JOIN
    project_name.dataset_name.concept concept
ON
    person.gender_concept_id = concept.concept_id
")

tb <- bq_project_query(project_name, sql)
bq_table_download(tb) %>%
    group_by(concept_name) %>%
    summarize(n())

concept_name,n()
<chr>,<int>
FEMALE,19189
MALE,22872
No matching concept,5


___Reflection Question:___ How does the sex distribution compare to what you often see in RCTs?<br>
___Answer:___ We found 54.37% Male, 45.62% Female and 0.01 not matching with the concept id. Thus us in line with some sources stating slight overpresentation by men in RCTs.

### 3.1.3 Look at the race and ethnicity distribution of our cohort
The structure for this query will be strikingly similar to the one you created above. Use the same CTE you previously generated to once again query against the `person` and `concept` tables.  This time however, you'll want to extract information about these people's race and ethnicity.  In the `person` table, there exist separate columns that store the concept_id's related to one's race and ethnicity. Your resulting table should have three columns: `person_id`, `race`, `ethnicity`.


Read the query into an R dataframe, and stratify by race and ethnicity. What does the race and ethnicity distribution of these patients look like?  

In [30]:
############################################################
##### YOUR SOLUTION GOES HERE
############################################################
sql = format.query("
WITH cohort AS (
    SELECT 
        DISTINCT measurement.person_id
    FROM 
        project_name.dataset_name.measurement measurement
    JOIN 
        project_name.dataset_name.condition_occurrence condition
    ON 
        measurement.person_id = condition.person_id
    WHERE
        measurement.measurement_concept_id = 3004410
        AND measurement.value_as_number >= 8
        AND condition.condition_concept_id = 4193704 
)

SELECT
    c.person_id,
    concept_race.concept_name race,
    concept_ethnicity.concept_name ethnicity
FROM
    cohort c
INNER JOIN
    project_name.dataset_name.person
USING
    (person_id)
INNER JOIN
    project_name.dataset_name.concept concept_race
ON
    person.race_concept_id = concept_race.concept_id
INNER JOIN
    project_name.dataset_name.concept concept_ethnicity
ON
    person.ethnicity_concept_id = concept_ethnicity.concept_id
")

tb <- bq_project_query(project_name, sql)
df <- bq_table_download(tb)
head(df)

# Perform a group by and summarize on both race and ethnicity to take a look at the racial/ethnic distribtion
# of this cohort.

bq_table_download(tb) %>%
    group_by(race) %>%
    summarize(n = n(), .groups = "drop")

bq_table_download(tb) %>%
    group_by(ethnicity) %>%
    summarize(n = n(), .groups = "drop")


person_id,race,ethnicity
<int>,<chr>,<chr>
30176021,Black or African American,Not Hispanic or Latino
30132780,Black or African American,Not Hispanic or Latino
31384050,Black or African American,Not Hispanic or Latino
31114880,Black or African American,Not Hispanic or Latino
30855304,Black or African American,Not Hispanic or Latino
30981018,Black or African American,Not Hispanic or Latino


race,n
<chr>,<int>
American Indian or Alaska Native,264
Asian,8363
Black or African American,3131
Native Hawaiian or Other Pacific Islander,1175
No matching concept,12926
White,16207


ethnicity,n
<chr>,<int>
Hispanic or Latino,10814
No matching concept,2262
Not Hispanic or Latino,28990


___Reflection Question:___ What does the race/ethnicity distribution look like? Who is over and or underrepresented? How often is race or ethnicity information unknown?  What does this tell you about the quality of race/ethnicity information in EHR data? <br>
___Answer:___ The following appears to be underrepresented: American Indian/Alaska Native, Native Hawaiian/Pacific Islander, and Black/African American.


## 3.2 Extract and examine patient comorbidity information
Patients in our cohort (by definition) all will have a concept_id that maps to ICD-9 code 250.00. It is often useful to examine what other kinds of comorbidities are present in your cohort of interest.

### 3.2.1 Find all condition names assigned to patients in our cohort
Using the CTE you've defined in the previous step, join to the `condition_occurrence` and `concept` tables to extract all unique `concept_name`s assigned to patients in your cohort.  The resulting table should have two columns: `person_id` and `concept_name`.  NOTE: this query may take a few minutes to run.

How many unique condition_names exist for patients across our cohort?

In [31]:
############################################################
##### YOU SOLUTION GOES HERE
############################################################
sql = format.query("
WITH cohort AS (
    SELECT 
        DISTINCT measurement.person_id
    FROM project_name.dataset_name.measurement measurement
    JOIN project_name.dataset_name.condition_occurrence condition ON measurement.person_id = condition.person_id
    WHERE
        measurement.measurement_concept_id = 3004410
        AND measurement.value_as_number >= 8
        AND condition.condition_concept_id = 4193704 
)

SELECT
    c.person_id id,
    concept.concept_name name
FROM cohort c
INNER JOIN project_name.dataset_name.condition_occurrence condition USING (person_id)
INNER JOIN project_name.dataset_name.concept concept ON condition.condition_concept_id = concept.concept_id
")

tb <- bq_project_query(project_name, sql)
df_conds <- bq_table_download(tb, n_max = 1000000) # Update made by TA: Josh. Plz don't modify :)

# How many distinct conditions?
df_conds %>%
    distinct(name) %>%
    nrow()

[1] 7848

### 3.2.2 Find the 10 conditions that occur in the most patients in our cohort
Note that this is subtly distinct from asking which codes appear most often in our cohort - the reason being that particular conditions can be mentioned multiple times throughout a patient's medical timeline.  

You could pretty easily answer this question with a couple dplyr commands on the dataframe you just read in, but since this is predominantly a SQL workshop, we'll ask that you find these codes using SQL.  If you want, you can sanity check the output of your SQL query by finding these codes with dplyr commands and ensuring they align.

As a helpful hint, you may find it useful to declare multiple CTEs in your query.  So far you've been defining a CTE that extracts patients meeting your definition of Type II diabetes.  A second CTE that stores a temporary result of the table you generated in the previous query may be useful.  You can define multiple CTEs via the following scheme.

```
WITH [expression_1] AS (
SOME SELECT STATEMENT - for example, your cohort
),

[expression_2] AS (
SOME OTHER SELECT STATEMENT - for example, your previous query
)

SELECT BLAH
    FROM BLAH_BLAH
    WHERE INTERESTING = 1

```

In [32]:
############################################################
##### YOUR SOLUTION HERE
############################################################
sql = format.query("
WITH cohort AS (
    SELECT 
        DISTINCT measurement.person_id
    FROM project_name.dataset_name.measurement measurement
    JOIN project_name.dataset_name.condition_occurrence condition ON measurement.person_id = condition.person_id
    WHERE
        measurement.measurement_concept_id = 3004410
        AND measurement.value_as_number >= 8
        AND condition.condition_concept_id = 4193704 
),

conditions AS (
    SELECT
        c.person_id id,
        concept.concept_name concept_name
    FROM cohort c
    INNER JOIN project_name.dataset_name.condition_occurrence condition USING (person_id)
    INNER JOIN project_name.dataset_name.concept concept ON condition.condition_concept_id = concept.concept_id
)

SELECT
    concept_name, 
    COUNT(id) AS num_persons_with_condition
FROM conditions
GROUP BY concept_name
ORDER BY num_persons_with_condition DESC
")

tb <- bq_project_query(project_name, sql)
df_top_conds <- bq_table_download(tb)

# Display Top 10 concepts and how many people in your cohort have them.
# TODO - your R code here.
df_top_conds %>%
    head(10) %>%
    # Optional: Format as a nice table with percentages
    mutate(
        percentage = round(num_persons_with_condition * 100 / sum(df_top_conds$num_persons_with_condition), 2),
        percentage = sprintf("%.2f%%", percentage)
    )


concept_name,num_persons_with_condition,percentage
<chr>,<int>,<chr>
Type 2 diabetes mellitus without complication,1018378,4.68%
Essential hypertension,947710,4.35%
Type 2 diabetes mellitus,835698,3.84%
Hyperlipidemia,580918,2.67%
Hyperglycemia due to type 2 diabetes mellitus,412540,1.90%
Complication due to diabetes mellitus,231745,1.06%
Transplanted lung present,180783,0.83%
End-stage renal disease,166433,0.76%
Transplanted kidney present,160250,0.74%


___Reflection Question:___ Which condition concepts are most commonly found in your cohort.  Does this make sense?<br>
__Answer:__ "Type 2 diabetes mellitus without complication" is the one commonly found in my cohort. It does make sense because the codition we created the cohort are specific to that desease (ICD-9 COde 250.00 and HbA1c measurement  ≥8 )

### Congrats!
You've completed the workshop, and now hopefully have a much better understanding of what it takes to be able to extract and manipulate EHR data stored formatted to the OMOP CDM.  

# Extras!

If you choose to use data in the OMOP Common Data Model for your own analysis, you will likely want to perform more queries than those we've described above. Here are a few more examples to help you build comfort with the OMOP CDM.

The teaching team is happy to answer questions and give you feedback on your work in this section.

## 4 Extract and examine lab data for patients in our cohort.
It may also be useful to get a better understanding of the kinds of labs that are most commonly ordered for patients in your cohort, and what the distribution of results look like. We'll run an analysis quite similar to what we just did with conditions above, except you'll be extracting lab orders and results instead of diagnoses.

### 4.1 Find the top 5 lab tests that are most frequently resulted for patients in your cohort
The structure of this query should be remarkably similar to what you did in the previous section.  As above, you'll have one CTE that references your cohort, and you'll find it useful to have a second CTE that extracts all lab tests with results that are not NULL.

The stucture of the second CTE should be as follows. To find lab tests, you'll want to join your cohort to the `measurement` table using `person_id`, and then join the result of this to the `concept` table on `measurement_concept_id = concept_id`.  Then make sure to filter for when `concept_class_id`="Lab Test", and when the `value_as_number` column from the `measurement` table is not NULL.  You'll want to be able to reference the following columns from this CTE: `person_id`, `concept_id`, `concept_name`.  

Finally, to find the top 5 most commonly ordered lab tests across patients in your cohort, you'll want to query from your second CTE, group by concept_id and concept_name, and count the distinct number of patients in each group.  You can use the `ORDER BY` SQL command followed by the `DESC` keyword to order the labs in descending order of how often they are tested across patients.  Use `LIMIT 5` to only query the top 5 most frequent labs.

As a sanity check, it would make sense for HbA1c to be in the top 5, as we required all patients in our cohort to have this measurement.

In [33]:
############################################################
##### YOUR SOLUTION GOES HERE
############################################################
sql = format.query("
WITH cohort AS (
    SELECT 
        DISTINCT measurement.person_id
    FROM project_name.dataset_name.measurement measurement
    JOIN project_name.dataset_name.condition_occurrence condition ON measurement.person_id = condition.person_id
    WHERE
        measurement.measurement_concept_id = 3004410
        AND measurement.value_as_number >= 8
        AND condition.condition_concept_id = 4193704 
),

all_labs AS (
    SELECT 
        DISTINCT c.person_id, 
        concept.concept_id, 
        concept.concept_name
    FROM 
        cohort c
        INNER JOIN project_name.dataset_name.measurement measurement USING (person_id)
        INNER JOIN project_name.dataset_name.concept concept ON measurement.measurement_concept_id = concept.concept_id
    WHERE 
        concept.concept_class_id='Lab Test' 
        AND measurement.value_as_number IS NOT NULL
)

SELECT
    concept_id, 
    concept_name, 
    COUNT(person_id) AS num_persons_with_labs
FROM all_labs
GROUP BY concept_id, concept_name
ORDER BY num_persons_with_labs
DESC LIMIT 5
")

tb <- bq_project_query(project_name, sql)
df_labs_top_5 <- bq_table_download(tb)
df_labs_top_5

concept_id,concept_name,num_persons_with_labs
<int>,<chr>,<int>
3004410,Hemoglobin A1c/Hemoglobin.total in Blood,42066
3016723,Creatinine [Mass/volume] in Serum or Plasma,38639
3013682,Urea nitrogen [Mass/volume] in Serum or Plasma,38559
3023103,Potassium [Moles/volume] in Serum or Plasma,38525
3014576,Chloride [Moles/volume] in Serum or Plasma,38506


___Reflection Question:___ Does it make sense that these lab measurements are most common in your patient cohort?<br>
___Answer:___ 

### 4.2 Summarize lab results
Find the min, max, and average lab result values for each of these five labs for each patient in your cohort. Your final table should be in long format (specified below). If a patient does not have a lab measurement for a particular lab, the corresponding row for that lab and patient will simply not exist in your final table.   

You'll want to again use the CTE you've defined that allows you to reference your patient cohort. Use this CTE to JOIN to the `measurement` table on `person_id`.  Filter for concept_ids that map to the 5 concept ids you've listed in the 3.1, and filter out rows where value_as_number is not NULL.  You can then group by `person_id`,`concept_name`, and create three seperate columns for max, min, and average of each of the five labs.

The resulting query should have 5 columns: `person_id`, `concept_name` (name of the lab), `min_value`, `max_value`, `avg_value`.  


In [34]:
############################################################
##### YOUR SOLUTION GOES HERE
############################################################
sql = format.query("
WITH cohort AS (
    SELECT 
        DISTINCT measurement.person_id
    FROM project_name.dataset_name.measurement measurement
    JOIN project_name.dataset_name.condition_occurrence condition ON measurement.person_id = condition.person_id
    WHERE
        measurement.measurement_concept_id = 3004410
        AND measurement.value_as_number >= 8
        AND condition.condition_concept_id = 4193704 
),

lab_values AS (
    SELECT
        c.person_id,
        concept.concept_id,
        concept.concept_name,
        measurement.value_as_number
    FROM cohort c
    INNER JOIN project_name.dataset_name.measurement measurement USING (person_id)
    INNER JOIN project_name.dataset_name.concept concept ON measurement.measurement_concept_id = concept.concept_id
    WHERE
        measurement.value_as_number IS NOT NULL
        AND concept.concept_id IN (3004410, 3016723, 3013682, 3023103, 3014576)
)

SELECT
    person_id,
    concept_name,
    MIN(value_as_number) as min_value,
    MAX(value_as_number) as max_value,
    AVG(value_as_number) as avg_value
FROM lab_values
GROUP BY person_id, concept_name
ORDER BY person_id, concept_name
")

tb <- bq_project_query(project_name, sql)
df_lab_results <- bq_table_download(tb)
head(df_lab_results, 20)

person_id,concept_name,min_value,max_value,avg_value
<int>,<chr>,<dbl>,<dbl>,<dbl>
29923143,Chloride [Moles/volume] in Serum or Plasma,97.00,106.00,102.7142857
29923143,Creatinine [Mass/volume] in Serum or Plasma,0.37,0.47,0.4328571
29923143,Hemoglobin A1c/Hemoglobin.total in Blood,7.50,11.00,8.9818182
29923143,Potassium [Moles/volume] in Serum or Plasma,3.80,4.50,4.1285714
29923143,Urea nitrogen [Mass/volume] in Serum or Plasma,5.00,22.00,9.1428571
29923265,Chloride [Moles/volume] in Serum or Plasma,104.00,104.00,104.0000000
29923265,Creatinine [Mass/volume] in Serum or Plasma,0.59,0.65,0.6300000
29923265,Hemoglobin A1c/Hemoglobin.total in Blood,6.00,10.20,7.5456522
29923265,Potassium [Moles/volume] in Serum or Plasma,4.10,4.10,4.1000000
